# Transfer learning
https://keras.io/guides/transfer_learning/

In [16]:
# setup
import numpy as np
import tensorflow as tf
from tensorflow import keras


instantiate base model with pre trained weights

In [17]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.


freeze base model

In [18]:
base_model.trainable = False


create new model on top

In [19]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


get dataset

In [20]:
batch_size = 32
img_height = 480
img_width = 640

In [35]:
train_ds = keras.utils.image_dataset_from_directory(
  "ordered_data",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = keras.utils.image_dataset_from_directory(
  "ordered_data",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6627 files belonging to 3 classes.
Using 5302 files for training.
Found 6627 files belonging to 3 classes.
Using 1325 files for validation.


In [36]:
class_names = train_ds.class_names
print(class_names)

['0', '1', '2']


resize images

In [37]:
# resize images
size = (150, 150)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, size), y))

train model on new data

In [24]:
new_dataset = train_ds
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=5, validation_data=val_ds, callbacks=[keras.callbacks.EarlyStopping(patience=3)])


Epoch 1/5
166/166 [==============================] - 142s 835ms/step - loss: -3.7907 - binary_accuracy: 0.5639 - val_loss: -10.2785 - val_binary_accuracy: 0.5909
Epoch 2/5
166/166 [==============================] - 137s 819ms/step - loss: -13.9102 - binary_accuracy: 0.5645 - val_loss: -20.5480 - val_binary_accuracy: 0.5857
Epoch 3/5
166/166 [==============================] - 137s 819ms/step - loss: -24.0196 - binary_accuracy: 0.5653 - val_loss: -31.6498 - val_binary_accuracy: 0.5887
Epoch 4/5
166/166 [==============================] - 128s 768ms/step - loss: -33.8732 - binary_accuracy: 0.5641 - val_loss: -42.5007 - val_binary_accuracy: 0.5894
Epoch 5/5
166/166 [==============================] - 129s 777ms/step - loss: -44.3816 - binary_accuracy: 0.5662 - val_loss: -52.9798 - val_binary_accuracy: 0.5872


test/evaluate

In [27]:
# use val dataset to test for now
test_loss, test_acc = model.evaluate(val_ds, verbose=2)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

42/42 - 27s - loss: -5.2980e+01 - binary_accuracy: 0.5872 - 27s/epoch - 638ms/step
Test accuracy: 0.5871698260307312
Test loss: -52.97981262207031


In [29]:
test_predictions = model.predict(val_ds)
print(test_predictions.shape)
print(test_predictions[0:10])


42/42 [==============================] - 29s 668ms/step
(1325, 1)


model summary

In [33]:

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 2,049
Non-trainable params: 20,861,480
_________________________________________________________________
